In [1]:
from __future__ import print_function
import sqlite3
import paho.mqtt.client as mqtt
import csv
import datetime
import os
import json
import requests
import sys
from kafka import KafkaProducer
from kafka import SimpleProducer, KafkaClient
#basepath = '/root/sftp/volumes/saqn/


In [2]:
kafkaHome = 'smartaqnet-dev01.teco.edu'

frostHome = 'smartaqnet-dev01.teco.edu'
#smartaqnetHome = 'localhost'

def on_connect(client, userdata, flags, rc):
     print('Starting to log data.')
     #client.subscribe("v1.0/Things")
     client.subscribe("v1.0/Datastreams")
     #client.subscribe("v1.0/Locations")
     #client.subscribe("v1.0/HistoricalLocations")
     #client.subscribe("v1.0/Sensors")
     #client.subscribe("v1.0/ObservedProperties")
     client.subscribe("v1.0/FeaturesOfInterest")
     client.subscribe("v1.0/Observations")

     if rc==0:
        print('connection established')

     else:
        print('Bad connection (return code = %d).' % rc)

def on_message(client, userdata, msg):
    payload = msg.payload
    topic = msg.topic.split('/')[-1]

    print("Receiving new message with topic: " + topic)
    #print(topic)
    #print(payload)

    data = json.loads(payload.decode("utf-8"))
    if(topic == "Observations"):
        iot_id = data['Datastream']['@iot.id']
    else:
        iot_id = data.get('@iot.id')
    #print(iot_id)
    producer.send(topic, key= str(iot_id), value=data)#msg.payload)
    producer.flush()


In [3]:
kafka = KafkaClient( kafkaHome +':9092')

#producer = SimpleProducer(kafka, async=True)
#producer.send_messages(b'my-topic', b'async message')
producer = KafkaProducer(bootstrap_servers=kafkaHome + ':9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

#producer.send('fizzbuzz', key= 'uid', value={'foo': 'bar'})

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(frostHome, 1883, 60)
client.loop_forever()

Starting to log data.
connection established
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new m

Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new

Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new

Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new

Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new message with topic: Observations
Receiving new

...

Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInteres

Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Rec

Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInteres

Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: FeaturesOfInterest
Receiving new message with topic: Observations
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Receiving new message with topic: Datastreams
Rece

KeyboardInterrupt: 

In [ ]:
producer.send('fizzbuzz', key= 'uid', value={'foo': 'bar'})

In [4]:
import paho.mqtt.client as paho
 
def on_connect(client, userdata, flags, rc):
    print("CONNACK received with code %d." % (rc))
 
client = paho.Client()
client.on_connect = on_connect
client.connect("smartaqnet.teco.edu", 8083)

0

In [14]:
#!/usr/bin/env python
import threading, logging, time
import multiprocessing

from kafka import KafkaConsumer, KafkaProducer

kafkaUrl = "smartaqnet-dev.teco.edu"
kafkaUrl = "smartaqnet.teco.edu"
class Producer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.stop_event = threading.Event()
        
    def stop(self):
        self.stop_event.set()

    def run(self):
        producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')

        while not self.stop_event.is_set():
            producer.send('Things', "TTT")
            producer.send('Things', b"\xc2Hola, mundo!")
            time.sleep(1)

        producer.close()

class Consumer(multiprocessing.Process):
    def __init__(self):
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        consumer = KafkaConsumer(bootstrap_servers= kafkaUrl + ':9092',
                                 auto_offset_reset='latest',
                                 consumer_timeout_ms=1000)
        consumer.subscribe(['Things'])

        while not self.stop_event.is_set():
            for message in consumer:
                print(message)
                if self.stop_event.is_set():
                    break

        consumer.close()


In [15]:
tasks = [
        Producer(),
        Consumer()
]
for t in tasks:
        t.start()

time.sleep(3)
for t in tasks:
        t.stop()

ConsumerRecord(topic=u'Things', partition=0, offset=87, timestamp=1525342996936, timestamp_type=0, key=None, value='TTT', checksum=None, serialized_key_size=-1, serialized_value_size=3)
ConsumerRecord(topic=u'Things', partition=0, offset=88, timestamp=1525342996937, timestamp_type=0, key=None, value='\xc2Hola, mundo!', checksum=None, serialized_key_size=-1, serialized_value_size=13)
ConsumerRecord(topic=u'Things', partition=0, offset=89, timestamp=1525342997938, timestamp_type=0, key=None, value='TTT', checksum=None, serialized_key_size=-1, serialized_value_size=3)
ConsumerRecord(topic=u'Things', partition=0, offset=90, timestamp=1525342997938, timestamp_type=0, key=None, value='\xc2Hola, mundo!', checksum=None, serialized_key_size=-1, serialized_value_size=13)


In [4]:
for t in tasks:
        t.stop()